"We run a Q&A forum with 27.007 members and would like to keep track of our members’ activity 
(i.e. how many posts/comments do they submit to our forum). We would like to send out a mail to 
those who are becoming less active on our platform, in the hope they start using it more."

Ok, So the goal is to improve customer activity. Normally we only needed to extract some data from the csv-files and store them in a database and that
was the whole assigment. But I've done that in an afternoon so that shit is done already. Let's do
some showoff expansions just omda het kan. 

Showoff idea's
- Run that shit on an AWS cluster
- Do some crazy machine learning shit
- Dive into the data a bit further to do some analysis 
    - I saw that the person_0 file has personal identifiable data. Might be interesting to 
    predict that for example mainly users of Firefox are inactive users.
    - Hypotheses we could test: 
        - Browser, male/female, location (and show off our knowledge of ip- addresses, we might even go completely crazy and use 
        a public API for this), age, creationDate
        - Mail-provider (and show off our knowledge of regexes
        - Time of day a comment is posted
    - Go completely nuts and use NLP on the content (maybe a bit overkill...)
- Instead of using a local database using one via AWS, also more realistic in an actual production environment.
       
 TODO: transfer to EMR want mijn computertje ontploft bijna (refactor to a .py file so it can be run on EMR)

In [145]:
from pyspark.sql import SparkSession


import mysql.connector 


database_password = input("Your database password please my good sir")

In [146]:

'''
Yeah, don't mind this. It is just for myself to check if I have the correct java version activated.

source ~/.bashrc
source ~/.bash_profile

uncomment the following line in /.bash_profile
 - export JAVA_HOME=$(/usr/libexec/java_home -v 1.8) pyspark
'''
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk1.8.0_241.jdk/Contents/Home'

import subprocess
subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT)


b'java version "1.8.0_241"\nJava(TM) SE Runtime Environment (build 1.8.0_241-b07)\nJava HotSpot(TM) 64-Bit Server VM (build 25.241-b07, mixed mode)\n'

In [147]:
spark = SparkSession.builder.appName("Basics").getOrCreate()


Join the tables in the file comment_hasCreator_person_0_0 with the data in the file comment_0_0 on the comment's id. This way you have per person (i.e. his/her id) the set of comments/posts that he/she has made and the corresponding date of that post.

    Use Spark's join functionality for this.

   

In [148]:
#Reading in data
comment_ids = spark.read.csv('../forum/comment_hasCreator_person_0_0.csv', sep='|', header=True,inferSchema=True)
comment_list = spark.read.csv('../forum/comment_0_0.csv', sep='|', header=True, inferSchema=True)
mail = spark.read.csv('../forum/person_email_emailaddress_0_0.csv', sep='|', header=True,inferSchema=True)
person = spark.read.csv('../forum/person_0_0.csv', sep='|', header=True,inferSchema=True)

In [149]:
comment_ids.printSchema()
comment_list.printSchema()
mail.printSchema()
person.printSchema()

root
 |-- Comment.id: long (nullable = true)
 |-- Person.id: long (nullable = true)

root
 |-- id: long (nullable = true)
 |-- creationDate: string (nullable = true)
 |-- locationIP: string (nullable = true)
 |-- browserUsed: string (nullable = true)
 |-- content: string (nullable = true)
 |-- length: integer (nullable = true)

root
 |-- Person.id: long (nullable = true)
 |-- email: string (nullable = true)

root
 |-- id: long (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthday: timestamp (nullable = true)
 |-- creationDate: string (nullable = true)
 |-- locationIP: string (nullable = true)
 |-- browserUsed: string (nullable = true)



In [150]:
# print("Length of comment: ", comment_ids.count())
# print("Length of comment_list: ", comment_list.count())
# print("Length of mail: ", mail.count())


In [151]:
'''

Lets look what's in this data. Uncomment all show() thingies when you want to
see what is in each Dataframe. For now I have commented out all these methods 
because they make our program run quite slowly and printing all these things 
out also makes our output very unclear.

'''


# comment_ids.show()
# comment_list.show()
# mail.show()

"\n\nLets look what's in this data. Uncomment all show() thingies when you want to\nsee what is in each Dataframe. For now I have commented out all these methods \nbecause they make our program run quite slowly and printing all these things \nout also makes our output very unclear.\n\n"

In [152]:
'''
Renaming column name where we are going to join on because spark does not 
like dots and capital letters that much
'''
comment_ids = comment_ids.withColumnRenamed('Comment.id','comment_id')
comment_ids = comment_ids.withColumnRenamed('Person.id','person_id')

mail = mail.withColumnRenamed('Person.id','person_id')

comment_list = comment_list.withColumnRenamed('id','comment_id')
comment_list = comment_list.withColumnRenamed('creationDate','creationDate_comment')


person = person.withColumnRenamed('id','person_id')
person = person.withColumnRenamed('creationDate','creationDate_person')

# comment_ids.show()

In [153]:
#Converting to appropriate datatypes

comment_ids = comment_ids.withColumn('comment_id', comment_ids['comment_id'].cast('int'))
comment_ids = comment_ids.withColumn('person_id', comment_ids['person_id'].cast('int'))

comment_list = comment_list.withColumn('comment_id', comment_list['comment_id'].cast('int'))
comment_list = comment_list.withColumn('creationDate_comment', comment_list['creationDate_comment'].cast('timestamp'))

mail = mail.withColumn('person_id', mail['person_id'].cast('int'))

person = person.withColumn('person_id', person['person_id'].cast('int'))
person = person.withColumn('creationDate_person', person['creationDate_person'].cast('timestamp'))
person = person.withColumn('birthday', person['birthday'].cast('date'))




In [154]:
comment_ids.show()
comment_list.show()
mail.show()

+----------+---------+
|comment_id|person_id|
+----------+---------+
|         1|    20605|
|         2|    11700|
|        87|     9357|
|        88|     3329|
|        89|     9357|
|        90|    14959|
|        91|    14959|
|        92|     9363|
|        93|    25890|
|        94|     1448|
|        95|     9357|
|        96|     9363|
|        97|     7997|
|        98|     7989|
|        99|    27589|
|       100|    29082|
|       101|     7997|
|       102|     7997|
|       103|    17781|
|       104|    17732|
+----------+---------+
only showing top 20 rows

+----------+--------------------+---------------+-----------------+--------------------+------+
|comment_id|creationDate_comment|     locationIP|      browserUsed|             content|length|
+----------+--------------------+---------------+-----------------+--------------------+------+
|         1|2019-08-12 07:20:...|    24.246.24.0|          Firefox|                 yes|     3|
|         2|2019-08-12 04:04:...|   19

In [135]:
#We only need the 'id' and 'creationDate' columns so we'll drop the rest

comment_list = comment_list.select('comment_id','creationDate_comment')

In [136]:
#Joining the tables together

first_join = comment_ids.join(comment_list, on=['comment_id'], how= 'inner')
# first_join.show()


join the previous result with the data in the file person_email_emailaddress_0_0 on the person's id.

As the file with emails is quite small, you could do the join by making a dictionary that maps each person id to one of its email addresses and broadcast that dictionary to each mapper and do the join yourself. Or you can again make use of Spark's join functionality.


In [137]:
second_join = first_join.join(mail, on=["person_id"], how='inner')
# second_join.show()


In [138]:
# I might have to change person and second_join here again

final_join = person.join(second_join, on=["person_id"], how='inner')

In [139]:
'''
Comment these lines if you don't want to get Rick rolled and learn the danger of running 
code without understanding what it does or reading the documentation.
'''

# import webbrowser
# webbrowser.open('https://www.youtube.com/watch?v=dQw4w9WgXcQ') 

"\nComment these lines if you don't want to get Rick rolled and learn the danger of running \ncode without understanding what it does or reading the documentation.\n"


Group this set by the person's email and month (in which the comment was created) and count the number of comments.
So per person you should now have a record in the form of Person.email|Month|Number of comments


In [140]:
# Lets first look what's exactly in this 'creationDate' column

# final_join.select('creationDate').show()
# final_join.select('creationDate').head(5)



In [141]:
'''
Extracting the month from the 'creationDate_comment' column. 
Please note that these are in numerical format (E.g. 1 = "January")
'''

from pyspark.sql.functions import month
final_join = final_join.withColumn("creationDate_comment_month", month(final_join['creationDate_comment']))
# final_join= final_join.withColumn('creationDate_DT',functions.to_date(final_join.creationDate))
# final_join.printSchema()
# final_join.show()

In [142]:
feature_group = ['email','creationDate_comment_month']
final_join_short = final_join.groupBy(feature_group).count().withColumnRenamed('count', 'Number_of_comments_that_month')
final_join = final_join.join(final_join_short, feature_group)\
    .orderBy(["email", 'creationDate_comment_month']).dropDuplicates()


In [143]:
# final_join.show()

In [144]:
'''
Ok, this is going to be hard but it will be worth it, pinky promise!

- Dowload the jbdc connector here (choose 'Platform independent'): https://dev.mysql.com/downloads/connector/j/
- Unpack the zip file

- sudo mv ~/Downloads/mysql-connector-java-8.0.19 /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pyspark/jars
(change according to your own locations of course)

if this is too hard: use the code from, much longer, solution called "old database" 
but be warned: collect() might not work if you use it on your complete dataset. 
'''

#This might take a few minutes to run... What we are doing now is pushing the whole dataset into our database. 

mode = 'overwrite'
properties = {
    "user": "root",
    "password": database_password,
    "driver": "com.mysql.jdbc.Driver"
}

final_join_short.write.jdbc(url='jdbc:mysql://localhost/forum?serverTimezone=UTC', table="mail", mode=mode, properties=properties)


Py4JJavaError: An error occurred while calling o1041.jdbc.
: java.sql.SQLException: Access denied for user 'root'@'localhost' (using password: NO)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:129)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:97)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:122)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:836)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:456)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:246)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:197)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverWrapper.connect(DriverWrapper.scala:45)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anonfun$createConnectionFactory$1.apply(JdbcUtils.scala:63)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anonfun

In [ ]:
# final_join.count()


In [ ]:
# Sampling Dataframe for data Analysis
sample = final_join.sample(0.01)

In [ ]:
# sample.count()


In [ ]:
sample.write.save("data/final_join.parquet", mode='overwrite')


In [ ]:
# checking if person_id can have multiple entries, which it can
sample.createOrReplaceTempView("table1")
spark.sql("select * from table1 where person_id = 15870").show()

'''
So this is stupid: our assignment specifies that we need to
"group this set by the person’s email and month", but we now
also know that a person can have multiple emailaddresses, 
which can screw up our interpretation. It would be better
to group on person_id

'''

That's it, We'll do our Data Analysis in another file.


